In [1]:
from typing import List, cast
import datetime
import pandas as pd
import logging 
import matplotlib.pyplot as plt
from sklearn.multioutput import RegressorChain
from xgboost import XGBRegressor
from data import SCRAPED_STOCK_FILE_PATH 
from gsp.model.utils import (
    MOVING_WINDOW_AGGREGATORS_ALIAS, 
    make_shift_in_groups, 
    make_mw_in_groups, 
    get_most_recent_working_date, 
    get_nth_previous_working_date,
    show,
    get_all_missing_stock_names,
    get_minimal_stocks_existence_date
)

logger = logging.getLogger(__name__)

GLOBAL PARAMETERS - SETTINGS

In [ ]:
# --- MAIN SETUP ---
MAIN_INDEX = ["Date", "Name"]
SECONDARY_INDEX = ["Name"]
YEARS_BACK_TO_CONSIDER = 5
N_STEP_PREDICTION = 15
DAYS_BACK_TO_TEST = 3*N_STEP_PREDICTION
TODAY = get_most_recent_working_date()
DATA_START_DATE = TODAY - datetime.timedelta(days=365*YEARS_BACK_TO_CONSIDER)


# --- TRAINING & TESTING SETUP ---
TRAINING_END_DATE = get_nth_previous_working_date(n=DAYS_BACK_TO_TEST, date=TODAY)
TESTING_START_DATE = get_nth_previous_working_date(n=-N_STEP_PREDICTION, date=TRAINING_END_DATE)
TESTING_END_DATE = get_nth_previous_working_date(n=N_STEP_PREDICTION, date=TODAY)

show(DATA_START_DATE, TRAINING_END_DATE, TESTING_START_DATE, TESTING_END_DATE, TODAY)


LOAD DATA

In [ ]:
stocks = (
    pd.read_csv(
        SCRAPED_STOCK_FILE_PATH,
        dtype={
            "Date": "period[D]",
            "Open": "float",
            "High": "float",
            "Low": "float",
            "Close": "float",
            "Volume": "int",
            "Area": "category",
            "Name": "category",
        },
        index_col=MAIN_INDEX,
    )
    .sort_index()
    .loc[DATA_START_DATE.isoformat():]
)

show(stocks)

PREPROCESSING

In [ ]:
def preprocessX(
    data: pd.DataFrame,
    shifts_list: List[int],
    windows_list: List[int],
    aggregators_list: List[MOVING_WINDOW_AGGREGATORS_ALIAS] | MOVING_WINDOW_AGGREGATORS_ALIAS,
) -> pd.DataFrame:
    data = data.copy()

    data["DayOfWeek"] = data.index.get_level_values("Date").dayofweek # type: ignore
    data["Month"] = data.index.get_level_values("Date").month # type: ignore
    data["Year"] = data.index.get_level_values("Date").year # type: ignore
    data["IsMonthStart"] = data.index.get_level_values("Date").day < 5 # type: ignore
    data["IsMonthEnd"] = data.index.get_level_values("Date").days_in_month - data.index.get_level_values("Date").day < 5 # type: ignore
    data["Quarter"] = data.index.get_level_values("Date").quarter # type: ignore
    data["NameCat"] = data.index.get_level_values("Name")
    # Prodcut lunches from those comapnies

    grouped_lags: List[pd.DataFrame | pd.Series] = [
        make_shift_in_groups(df=data, groupby=SECONDARY_INDEX, column="Close", shift=shifts_list),
        make_mw_in_groups(df=data, groupby=SECONDARY_INDEX, column="Close", window=windows_list, aggregator=aggregators_list),
    ]

    data_lag = data.join(grouped_lags)

    categorical_features = ["Area","NameCat","DayOfWeek", "Month", "Year","IsMonthStart", "IsMonthEnd", "Quarter"]

    data_encoded = pd.get_dummies(data_lag, columns=categorical_features, drop_first=True)

    return data_encoded.drop(columns=["Open", "High", "Low", "Adj Close"])


def preprocessY(data: pd.DataFrame) -> pd.DataFrame:
    data = data.copy()
    return make_shift_in_groups(df = data, groupby=SECONDARY_INDEX, column="Close", shift=[-i for i in range(1,N_STEP_PREDICTION+1)])

X = preprocessX(
    stocks, 
    shifts_list=[1, 2, 3, 4, 8, 16, 24, 32], 
    windows_list=[4, 8, 16, 24, 32], 
    aggregators_list="mean"
)
y = preprocessY(stocks)

show(X, y)

SPLITTING DATA

In [ ]:
X_train, y_train = (
    X.loc[:TRAINING_END_DATE.isoformat()], 
    y.loc[:TRAINING_END_DATE.isoformat()]
)
X_test, y_test = (
    X.loc[TESTING_START_DATE.isoformat():TESTING_END_DATE.isoformat()], 
    y.loc[TESTING_START_DATE.isoformat():TESTING_END_DATE.isoformat()]
) 

show(X_train, y_train, X_test, y_test)


MODELLING

In [ ]:
def create_model(**hyper_params) -> RegressorChain:
    model = RegressorChain(XGBRegressor(**hyper_params))
    return model 

model = create_model()

TRAININING & TESTING

In [ ]:
logger.info("Fitting the model")
model = model.fit(X_train, y_train)
logger.info("Model fitted")